In [1]:
import pandas as pd
import plotly.graph_objects as go
import plotly_express as px

event = pd.read_csv('event.csv')
distribution_center = pd.read_csv('distribution_center.csv')
inventory_items = pd.read_csv('inventory_items.csv')
order_items = pd.read_csv('order_items.csv')
orders = pd.read_csv('orders.csv')
users = pd.read_csv('users.csv', low_memory=False)
products = pd.read_csv('products.csv')

In [2]:
# 첫 구매 상품의 카테고리가 일회성 소비 성격일 경우, 재구매가 적을 것이다


In [3]:
# 특정 브랜드만 구매만 하는 유저는 재구매가 적을 것이다.

order_items_products = pd.merge(order_items, products[['product_id', 'brand']], 
                                how='left', on='product_id')

valid_order = order_items_products[~order_items_products.status.isin(['Cancelled', 'Returned'])]

brand_cnt = valid_order.groupby('user_id').brand.nunique().to_frame('brand_cnt').reset_index()

order_items_products = pd.merge(order_items_products, brand_cnt, on='user_id', how='left')

# 브랜드 개수 1개 vs 2개 이상

# 유저별 구매 건수 계산
purchase_cnt = valid_order.groupby('user_id').size().to_frame('purchase_cnt').reset_index()

# nan인 경우 -> 취소나 반품만 한 유저저
order_items_products = pd.merge(order_items_products, purchase_cnt, on='user_id', how='left')
order_items_products[order_items_products.purchase_cnt.notna()]

order_items_products['group'] = order_items_products['brand_cnt'].apply(lambda x: '단일 브랜드' if x==1 else '여러 브랜드')
order_items_products['repurchase_cnt'] = order_items_products['purchase_cnt'] - 1

from scipy.stats import ttest_ind

single = order_items_products[order_items_products['group'] == '단일 브랜드']['repurchase_cnt']
multi = order_items_products[order_items_products['group'] == '여러 브랜드']['repurchase_cnt']

tstat, pval = ttest_ind(single, multi, equal_var=False)
print(f't-statistic: {tstat:.3f}, p-value: {pval:.3f}')

print('단일 브랜드 유저 수:', single.shape[0])
print('여러 브랜드 유저 수:', multi.shape[0])
print('단일 브랜드 repurchase_cnt 통계:\n', single.describe())
print('여러 브랜드 repurchase_cnt 통계:\n', multi.describe())


t-statistic: nan, p-value: nan
단일 브랜드 유저 수: 42140
여러 브랜드 유저 수: 139608
단일 브랜드 repurchase_cnt 통계:
 count    42140.000000
mean         0.007262
std          0.085740
min          0.000000
25%          0.000000
50%          0.000000
75%          0.000000
max          2.000000
Name: repurchase_cnt, dtype: float64
여러 브랜드 repurchase_cnt 통계:
 count    117398.000000
mean          2.588954
std           1.634755
min           0.000000
25%           1.000000
50%           2.000000
75%           3.000000
max          12.000000
Name: repurchase_cnt, dtype: float64


In [4]:
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

single_success = (single.values>0).sum()
single_total = single.shape[0]

multi_success = (multi.values > 0).sum()
multi_total = multi.shape[0]

count = np.array([single_success, multi_success])
nobs = np.array([single_total, multi_total])

stat, pval = proportions_ztest(count, nobs)
print('z-test 통계량:', stat)
print('p-value:', pval)

z-test 통계량: -313.83954266041377
p-value: 0.0


In [5]:
brand_hyphothesis = order_items_products.groupby('group').repurchase_cnt.mean().to_frame('repurchase_cnt').reset_index()

fig = px.bar(brand_hyphothesis, x='group', y='repurchase_cnt', template='simple_white',
             text_auto='.3f')

fig.update_layout(title=dict(text='단일 브랜드 vs 여러 브랜드 구매한 사용자의 재구매 횟수 비교',
                             x=0.5),
                xaxis_title='구매한 브랜드 개수',
                yaxis_title='재구매 횟수')
fig.show()

In [6]:
# 첫 구매 상품의 금액이 낮을수록 재구매가 적을 것이다

# 유저별 첫 주문 추출
first_orders = orders.sort_values(['user_id', 'created_at']).drop_duplicates('user_id', keep='first')[['user_id', 'order_id']]

# 첫 주문 금액 계산
first_order_items = pd.merge(first_orders, order_items[['order_id', 'sale_price']], on='order_id')
first_order_total = first_order_items.groupby('user_id')['sale_price'].sum().reset_index().rename(columns={'sale_price': 'first_order_total'})

# 유저별 전체 구매 횟수 계산
purchase_events = event[event['event_type'] == 'purchase']
purchase_event_counts = purchase_events.groupby('user_id').size().reset_index(name='purchase_event_count')

# 유저별 첫 주문 금액과 재구매 횟수 병합
merged = pd.merge(first_order_total, purchase_event_counts, on='user_id', how='left')
merged['purchase_event_count'] = merged['purchase_event_count'].fillna(0)  # 구매 기록 없는 사람은 0

# 첫 구매를 제외한 재구매 횟수로 변환
merged['repurchase_count'] = merged['purchase_event_count'] - 1
merged['repurchase_count'] = merged['repurchase_count'].clip(lower=0)  # 음수 방지

def first_segment(x):
    if x<=50:
        return '~$50'
    elif x<=100:
        return '$50~$100'
    elif x<=200:
        return '$100~$200'
    elif x<=300:
        return '$200~$300'
    elif x<=500:
        return '$300~$500'
    else:
        return '$500~'
    
merged['amount_segment'] = merged.first_order_total.apply(first_segment)

first_df = merged.groupby('amount_segment').repurchase_count.mean().to_frame('repurchase_mean').reset_index()

amount_order = ['~$50', '$50~$100', '$100~$200', '$200~$300', '$300~$500', '$500~']

first_df['amount_segment'] = pd.Categorical(first_df['amount_segment'], categories=amount_order, ordered=True)
first_df = first_df.sort_values('amount_segment')
first_df

fig = px.histogram(first_df, x='amount_segment', y='repurchase_mean', text_auto='.2f')
fig.update_layout(title=dict(text='첫 구매 금액에 따른 재구매 횟수 평균 비교',
                             x=0.5),
                xaxis_title='첫 구매 금액',
                yaxis_title='재구매 횟수 평균',
                template='simple_white')
fig.show()

In [7]:
from scipy.stats import f_oneway

groups = []
for segment in merged['amount_segment'].unique():
    group = merged[merged['amount_segment'] == segment]['repurchase_count']
    groups.append(group)

f_stat, p_value = f_oneway(*groups)
print(f'F-statistic: {f_stat:.4f}, p-value: {p_value:.4f}')


F-statistic: 1440.0426, p-value: 0.0000


In [8]:
# 첫 구매 후 장바구니/조회 이벤트 수가 적은 유저는 재구매가 낮을 것이다

event_count = event[event['user_id'] != -1].groupby('user_id').size()
avg_event_count = event_count.mean()

high_event_users = event_count[event_count >= avg_event_count].index
low_event_users = event_count[event_count < avg_event_count].index

purchases = event[(event['event_type'] == 'purchase') & (event['user_id'] != -1)]
purchase_users = purchases['user_id'].unique()

high_total = len(high_event_users)
high_bought = len(set(high_event_users) & set(purchase_users))

low_total = len(low_event_users)
low_bought = len(set(low_event_users) & set(purchase_users))

high_group_purchases = purchases[purchases['user_id'].isin(high_event_users)].groupby('user_id').size().mean()
low_group_purchases = purchases[purchases['user_id'].isin(low_event_users)].groupby('user_id').size().mean()

print(f'High Event 유저 평균 재구매 횟수: {high_group_purchases-1:.2f}')
print(f'Low Event 유저 평균 재구매 횟수: {low_group_purchases-1:.2f}')

fig = go.Figure(data=[
    go.Bar(name='이벤트 많은 유저', x=['이벤트 많은 유저 평균'], y=[high_group_purchases-1], text=(high_group_purchases-1).round(2)),
    go.Bar(name='이벤트 적은 유저', x=['이벤트 적은 유저 평균'], y=[low_group_purchases-1], text=(low_group_purchases-1).round(2))
])

fig.update_layout(
    title=dict(text='이벤트 수에 따른 평균 재구매 횟수 비교', x=0.5),
    yaxis_title='평균 재구매 횟수',
    xaxis_title='User Group',
    bargap=0.5,
    template='simple_white'
)

fig.show()

High Event 유저 평균 재구매 횟수: 3.32
Low Event 유저 평균 재구매 횟수: 0.41


In [9]:
from scipy.stats import ttest_ind

# 각 유저별 재구매 횟수 리스트
high_repurchase_counts = purchases[purchases['user_id'].isin(high_event_users)].groupby('user_id').size() - 1
low_repurchase_counts = purchases[purchases['user_id'].isin(low_event_users)].groupby('user_id').size() - 1

# t-test
t_stat, p_value = ttest_ind(high_repurchase_counts, low_repurchase_counts, equal_var=False)  # equal_var=False는 Welch’s t-test

print(f't-statistic: {t_stat:.4f}')
print(f'p-value: {p_value:.4f}')


t-statistic: 301.3477
p-value: 0.0000


In [10]:
# 첫 구매 후 일정 기간 내에 사이트를 다시 방문한 고객일 수록 재구매 횟수가 많많을 것이다

# 1. 유저별 첫 구매일 계산
first_purchase = orders.groupby('user_id')['created_at'].min().reset_index()
first_purchase.columns = ['user_id', 'first_purchase_date']

# 2. 방문 이벤트 (user_id != -1) & (특정 event_type)
visit_events = event[(event.user_id != -1) & (event['event_type'].isin(['product', 'department', 'home']))]

# 3. 첫 구매 이후 재방문
visit_after_purchase = visit_events.merge(first_purchase, on='user_id')
visit_after_purchase = visit_after_purchase[visit_after_purchase['created_at'] > visit_after_purchase['first_purchase_date']]

# 4. 유저별 첫 재방문일
first_revisit = visit_after_purchase.groupby('user_id')['created_at'].min().reset_index()
first_revisit.columns = ['user_id', 'first_revisit_date']

# 5. 유저별 재구매 횟수
order_counts = orders.groupby('user_id').size().reset_index(name='order_count')
order_counts['repurchase_count'] = (order_counts['order_count'] - 1).clip(lower=0)

# 6. 분석 테이블 결합
analysis_df = first_purchase.merge(first_revisit, on='user_id', how='left') \
                             .merge(order_counts[['user_id', 'repurchase_count']], on='user_id', how='left')

# 7. 날짜 차이 계산
analysis_df['first_purchase_date'] = pd.to_datetime(analysis_df['first_purchase_date'])
analysis_df['first_revisit_date'] = pd.to_datetime(analysis_df['first_revisit_date'])
analysis_df['days_to_revisit'] = (analysis_df['first_revisit_date'] - analysis_df['first_purchase_date']).dt.days

def revisit_segment(x):
    if pd.isna(x):
        return 'No Revisit'
    elif x <= 3:
        return '0-3 days'
    elif x <= 7:
        return '4-7 days'
    elif x <= 14:
        return '8-14 days'
    elif x <= 30:
        return '15-30 days'
    else:
        return '31+ days'

analysis_df['revisit_group'] = analysis_df['days_to_revisit'].apply(revisit_segment)



# 그룹별 재구매 "횟수" 평균 보기
revisit_stats = analysis_df.groupby('revisit_group')['repurchase_count'].mean().reset_index()
revisit_order = ['No Revisit', '0-3 days', '4-7 days', '8-14 days', '15-30 days', '31+ days']

revisit_stats['revisit_group'] = pd.Categorical(revisit_stats['revisit_group'], categories=revisit_order, ordered=True)
revisit_stats = revisit_stats.sort_values('revisit_group')
revisit_stats

,revisit_group,repurchase_count
5,No Revisit,0.014708
0,0-3 days,0.852964
3,4-7 days,1.661479
4,8-14 days,1.638298
1,15-30 days,1.629004
2,31+ days,1.475158


In [11]:
analysis_df.days_to_revisit.isna().sum()

48612

In [12]:
fig = px.histogram(revisit_stats, x='revisit_group', y='repurchase_count',
                   text_auto='.1f')

fig.update_layout(title=dict(text='첫 구매 후 재방문 기간별 재구매 횟수 평균 비교',
                  x=0.5),
                  xaxis_title='재방문 기간',
                  yaxis_title='재구매 횟수 평균',
                  template='simple_white')

fig.show()

In [13]:
from scipy.stats import f_oneway

# 그룹별로 repurchase_count 리스트 만들기
groups = [group['repurchase_count'].dropna().values for name, group in analysis_df.groupby('revisit_group')]

# ANOVA 검정
f_stat, p_value = f_oneway(*groups)

print(f"F-statistic: {f_stat:.4f}")
print(f"P-value: {p_value:.4f}")


F-statistic: 29722.1137
P-value: 0.0000
